In [29]:
import pandas as pd
index1 = pd.MultiIndex.from_tuples([
    ('California', 2000), ('California', 2010), ('California', 2020),
    ('New York', 2000), ('New York', 2010), ('New York', 2020),
    ('Texas', 2000), ('Texas', 2010) , ('Texas', 2020)])

index2 = pd.MultiIndex.from_tuples([
    ('California', 2000), ('California', 2010),
    ('New York', 2000), ('New York', 2010),
    ('Texas', 2000), ('Texas', 2010)])

# Columns as requested
columns_df1 = ['Downtown', 'Belt', 'Outskirts']
columns_df2 = ['Belt', 'Outskirts', 'Land']

# Data for df1
data_df1 = [[1000000, 500000, 200000], 
            [1000010, 500010, 200010], 
            [1000020, 500020, 200020],
            [5000000, 2000000, 1000000], 
            [5000010, 2000010, 1000010], 
            [5000020, 2000020, 1000020],
            [3000000, 900000, 700000], 
            [3000010, 900010, 700010], 
            [3000020, 900020, 700020],]

# Data for df2
data_df2 = [[500000, 200000, 100000], 
            [500010, 200010, 100010], 
            [2000000, 1000000, 500000], 
            [2000010, 1000010, 500010], 
            [900000, 700000, 300010], 
            [900010, 700010, 300010],]

# Creating DataFrames
df1 = pd.DataFrame(data_df1, index=index1, columns=columns_df1)
df2 = pd.DataFrame(data_df2, index=index2, columns=columns_df2)

In [37]:
df_new = pd.concat([df1, df2], sort=True)
df_new

Belt   Downtown      Land  Outskirts
California 2000   500000  1000000.0       NaN     200000
           2010   500010  1000010.0       NaN     200010
           2020   500020  1000020.0       NaN     200020
New York   2000  2000000  5000000.0       NaN    1000000
           2010  2000010  5000010.0       NaN    1000010
           2020  2000020  5000020.0       NaN    1000020
Texas      2000   900000  3000000.0       NaN     700000
           2010   900010  3000010.0       NaN     700010
           2020   900020  3000020.0       NaN     700020
California 2000   500000        NaN  100000.0     200000
           2010   500010        NaN  100010.0     200010
New York   2000  2000000        NaN  500000.0    1000000
           2010  2000010        NaN  500010.0    1000010
Texas      2000   900000        NaN  300010.0     700000
           2010   900010        NaN  300010.0     700010

# AI supported Time-Series Analysis

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import datetime

pd.set_option('display.max_rows', 10)

## NeuNet Class

In [3]:
class NeuNet(object):                                                               # to the class we shall provide a model, a loss_fn and an optimizer.
    def __init__(self, model, loss_fn, optimizer):
        # Here we define the attributes of our class
        
        # We start by storing the arguments as attributes to use them later
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        # Let's send the model to the specified device right away
        self.model.to(self.device)                                                  # here we send the model to the device

        # These attributes are defined here, but since they are
        # not informed at the moment of creation, we keep them None
        self.train_loader = None
        self.val_loader = None
        self.writer = None
        
        # These attributes are going to be computed internally
        self.losses = []
        self.val_losses = []
        self.total_epochs = 0

        # Creates the train_step function for our model, 
        # loss function and optimizer
        # Note: there are NO ARGS there! It makes use of the class
        # attributes directly
        self.train_step_fn = self._make_train_step_fn()
        # Creates the val_step function for our model and loss
        self.val_step_fn = self._make_val_step_fn()

    def to(self, device):                                                           # this is the function sending the model to the device
        # This method allows the user to specify a different device
        # It sets the corresponding attribute (to be used later in
        # the mini-batches) and sends the model to the device
        try:
            self.device = device
            self.model.to(self.device)
        except RuntimeError:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            print(f"Couldn't send it to {device}, sending it to {self.device} instead.")
            self.model.to(self.device)

    def set_loaders(self, train_loader, val_loader=None):                           # data loaders provide the input data in a sutiable format to the model, in a minibatch size
        # This method allows the user to define which train_loader (and val_loader, optionally) to use
        # Both loaders are then assigned to attributes of the class
        # So they can be referred to later
        self.train_loader = train_loader
        self.val_loader = val_loader

    def set_tensorboard(self, name, folder='runs'):
        # This method allows the user to define a SummaryWriter to interface with TensorBoard
        suffix = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
        self.writer = SummaryWriter(f'{folder}/{name}_{suffix}')

    def _make_train_step_fn(self):
        # This method does not need ARGS... it can refer to
        # the attributes: self.model, self.loss_fn and self.optimizer
        
        # Builds function that performs a step in the train loop
        def perform_train_step_fn(x, y):
            # Sets model to TRAIN mode
            self.model.train()                                                      # the model has a different behaviour during training and evaluation mode

            # Step 1 - Computes our model's predicted output - forward pass
            yhat = self.model(x)
            # Step 2 - Computes the loss
            loss = self.loss_fn(yhat, y)
            # Step 3 - Computes gradients for both "a" and "b" parameters
            loss.backward()
            # Step 4 - Updates parameters using gradients and the learning rate
            self.optimizer.step()
            self.optimizer.zero_grad()                                              # avoid cumulation of gradients

            # Returns the loss
            return loss.item()

        # Returns the function that will be called inside the train loop
        return perform_train_step_fn
    
    def _make_val_step_fn(self):
        # Builds function that performs a step in the validation loop
        def perform_val_step_fn(x, y):
            # Sets model to EVAL mode
            self.model.eval()                                                       # here we set the model to evaluation mode

            # Step 1 - Computes our model's predicted output - forward pass
            yhat = self.model(x)
            # Step 2 - Computes the loss
            loss = self.loss_fn(yhat, y)
            # There is no need to compute Steps 3 and 4, 
            # since we don't update parameters during evaluation
            return loss.item()

        return perform_val_step_fn
            
    def _mini_batch(self, validation=False):
        # The mini-batch can be used with both loaders
        # The argument `validation`defines which loader and 
        # corresponding step function is going to be used
        if validation:
            data_loader = self.val_loader
            step_fn = self.val_step_fn
        else:
            data_loader = self.train_loader
            step_fn = self.train_step_fn

        if data_loader is None:
            return None
            
        # Once the data loader and step function, this is the 
        # same mini-batch loop we had before
        mini_batch_losses = []
        for x_batch, y_batch in data_loader:
            x_batch = x_batch.to(self.device)
            y_batch = y_batch.to(self.device)

            mini_batch_loss = step_fn(x_batch, y_batch)
            mini_batch_losses.append(mini_batch_loss)

        loss = np.mean(mini_batch_losses)
        return loss

    def set_seed(self, seed=42):
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False    
        torch.manual_seed(seed)
        np.random.seed(seed)
    
    def train(self, n_epochs, seed=42):                                             # this function execute the training of the model
        # To ensure reproducibility of the training process
        self.set_seed(seed)

        for epoch in tqdm(range(n_epochs)):
            # Keeps track of the numbers of epochs
            # by updating the corresponding attribute
            self.total_epochs += 1

            # inner loop
            # Performs training using mini-batches
            loss = self._mini_batch(validation=False)
            self.losses.append(loss)

            # VALIDATION
            # no gradients in validation!
            with torch.no_grad():
                # Performs evaluation using mini-batches
                val_loss = self._mini_batch(validation=True)
                self.val_losses.append(val_loss)

            # If a SummaryWriter has been set...
            if self.writer:                                                         # this is optional, i.e. Tensorboard output
                scalars = {'training': loss}
                if val_loss is not None:
                    scalars.update({'validation': val_loss})
                # Records both losses for each epoch under the main tag "loss"
                self.writer.add_scalars(main_tag='loss',
                                        tag_scalar_dict=scalars,
                                        global_step=epoch)

        if self.writer:
            # Closes the writer
            self.writer.close()

    def save_checkpoint(self, filename):
        # Builds dictionary with all elements for resuming training
        checkpoint = {'epoch': self.total_epochs,
                      'model_state_dict': self.model.state_dict(),
                      'optimizer_state_dict': self.optimizer.state_dict(),
                      'loss': self.losses,
                      'val_loss': self.val_losses}

        torch.save(checkpoint, filename)

    def load_checkpoint(self, filename):
        # Loads dictionary
        checkpoint = torch.load(filename)

        # Restore state for model and optimizer
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        self.total_epochs = checkpoint['epoch']
        self.losses = checkpoint['loss']
        self.val_losses = checkpoint['val_loss']

        self.model.train() # always use TRAIN for resuming training   

    def predict(self, x):
        # Set is to evaluation mode for predictions
        self.model.eval() 
        # Takes aNumpy input and make it a float tensor
        x_tensor = torch.as_tensor(x).float()
        # Send input to device and uses model for prediction
        y_hat_tensor = self.model(x_tensor.to(self.device))                                 # sending input to device
        # Set it back to train mode
        self.model.train()
        # Detaches it, brings it to CPU and back to Numpy
        return y_hat_tensor.detach().cpu().numpy()                                          # sending back to cpu for return

    def plot_losses(self):
        fig = plt.figure(figsize=(10, 4))
        plt.plot(self.losses, label='Training Loss', c='b', lw=1)
        plt.plot(self.val_losses, label='Test Loss', c='r', lw=1)
        plt.yscale('log')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.tight_layout()
        return fig

    def add_graph(self):
        # Fetches a single mini-batch so we can use add_graph
        if self.train_loader and self.writer:
            x_sample, y_sample = next(iter(self.train_loader))
            self.writer.add_graph(self.model, x_sample.to(self.device))


# Pytorch Neural Network 

## Loading and Transforming Data

In [5]:
# PATH_NETWORK_WG = r".\network_with_gen.csv" 
# df_I = pd.read_csv(PATH_NETWORK_WG, sep=";", decimal=",")
# temp_df = df_I.copy()
# arr = temp_df.to_numpy()
# df_I = pd.DataFrame(np.tile(arr, (240, 1)), columns = temp_df.columns)

PATH_MONTECARLO = r".\montecarlo_database.csv" 
df_montecarlo = pd.read_csv(PATH_MONTECARLO, sep=",", index_col=[0, 1, 2, 3, 4])# , decimal=",")
column_mapping = {col: i+1 for i, col in enumerate(df_montecarlo.columns)}
timestep_mapping_df = pd.DataFrame(list(column_mapping.items()), columns=['time', 'timestep'])
# number_of_strata = df_montecarlo['strata'].max()
# number_of_iterations = df_montecarlo['iteration'].max()+1

PATH_NETWORK = r".\network.xlsx" 
df_network = pd.read_excel(PATH_NETWORK, sheet_name="profiles", decimal=",")
df_network = df_network.drop(index=0).reset_index(drop=True)
df_network = df_network.drop(df_network.columns[0], axis=1)
temp_df = df_network.copy()
arr = temp_df.to_numpy()
arr = arr.astype(np.float64)
df_network = pd.DataFrame(np.tile(arr, (240, 1)), columns = temp_df.columns)

PATH_ENGINE = r".\engine_database.csv"
df_engine = pd.read_csv(PATH_ENGINE, sep=",", index_col=[0, 1, 2, 3, 4])

In [ ]:
#CURRENTLY RECODED !!!
X_df = df_engine.loc[:,:,"in_service","line",:,:].stack().unstack("id")
number_of_lines = len(set(X_df.columns)) # number of lines
df_network.index= X_df.index
X_df = pd.concat([X_df, df_network], axis=1)
###
# Now the engine database is needed!
###
y_df = df_engine.loc[:,:,"loss_of_load_p_mw","load",:,:].stack().unstack("id")
ysum_df = pd.DataFrame(y_df.sum(axis=1))

In [6]:
X_df_montecarlo = df_montecarlo.stack().unstack("id")
df_network.index= X_df_montecarlo.index
number_of_lines = len(set(X_df_montecarlo.columns))
X_df_montecarlo = pd.concat([X_df_montecarlo, df_network], axis=1)
X_df_montecarlo.insert(0, 'idx', range(1, len(X_df_montecarlo) + 1))
X = X_df_montecarlo.to_numpy()


z = pd.DataFrame(X).iloc[:,1:number_of_lines+1].astype(int).astype(str).agg(''.join, axis=1)
l = []
from collections import Counter
c = Counter(z)
for k in z:
    if c[k] == 1:
        l.append(str('G0'))
    else:
        l.append(str(k))

X_train, X_val = train_test_split(X, train_size = 0.1, stratify = pd.DataFrame(l)[0], shuffle = True, random_state = 42)
        
###
# Now the engine database is needed!
# Step 1: Montecarlo Sampling
# Step 2: Run OPF for selected 10% Finished!
# Step 3: Populate Engine Database
###

# y_df = df_engine.loc[:,:,"loss_of_load_p_mw","load",:,:].stack().unstack("id")
# ysum_df = pd.DataFrame(y_df.sum(axis=1))

In [ ]:
# OLD CODE, KEEP FOR SAFETY
idx_for_opf = pd.Series(pd.DataFrame(X_train)[0])
opfs_interval = (
    X_df_montecarlo[X_df_montecarlo['idx'].isin(idx_for_opf)]
    .reset_index()
    .rename(columns={'level_4': 'time'})
    [['iteration', 'time']]
    .merge(timestep_mapping_df, on='time', how='left')
)

for _,row in opfs_interval[['iteration','timestep']].iterrows():
    iterationSet = row['iteration']
    timestep = row['timestep']

opfs_interval[['iteration','timestep']]
grouped_list = opfs_interval.groupby('iteration')['timestep'].apply(list).reset_index()
grouped_list


In [7]:
idx_for_opf = pd.Series(X_train[:, 0])  # Assuming X_train is a NumPy array. If not, the original code works.

opfs_interval = (
    X_df_montecarlo[X_df_montecarlo['idx'].isin(idx_for_opf)]
    .reset_index()
    .rename(columns={'level_4': 'time'})
    [['iteration', 'time']]
    .merge(timestep_mapping_df, on='time', how='left')
)

grouped_list = (
    opfs_interval.groupby('iteration')['timestep']
    .apply(list)
    .reset_index()
)

grouped_list


,iteration,timestep
0,0,"[1, 3, 13, 17, 24, 39, 42, 43, 45, 49]"
1,1,"[13, 50]"
2,2,"[12, 16, 27, 36, 43, 47, 49]"
3,3,"[1, 2, 20, 22, 33, 37, 49]"
4,4,"[20, 23, 25, 29, 38]"
...,...,...
233,235,"[20, 36, 37, 41, 47]"
234,236,"[2, 28, 39]"
235,237,"[10, 22, 24, 35, 38, 48]"
236,238,"[9, 14, 17, 23, 26, 37]"


In [21]:
iteration_to_timestep = {row['iteration']: row['timestep'] for _, row in grouped_list.iterrows()}
iteration_to_timestep[240]

KeyError: 240

In [15]:
for  _, row in grouped_list.iterrows():
    print(f"Iteration {row['iteration']}: {row['timestep']}")
    print("--")

Iteration 0: [2, 4, 50]
--
Iteration 1: [13, 50]
--
Iteration 2: [12, 16, 27, 36, 43, 47, 49]
--
Iteration 3: [1, 2, 20, 22, 33, 37, 49]
--
Iteration 4: [20, 23, 25, 29, 38]
--
Iteration 5: [2, 4, 11, 27, 42, 43]
--
Iteration 7: [4, 16, 31, 43]
--
Iteration 8: [16, 20, 21, 28, 38, 47, 50]
--
Iteration 9: [15, 19, 20, 23, 34, 41, 43]
--
Iteration 10: [23, 33, 34]
--
Iteration 11: [1, 9, 11, 31, 39]
--
Iteration 12: [13, 22]
--
Iteration 13: [20, 21, 22, 26, 32, 39, 49]
--
Iteration 14: [8, 15, 22, 29, 40]
--
Iteration 15: [4, 13, 17, 43]
--
Iteration 16: [1, 9, 13, 17, 37]
--
Iteration 17: [17, 32, 34, 48]
--
Iteration 18: [14, 19, 26, 33, 48]
--
Iteration 19: [24]
--
Iteration 20: [13, 20, 35]
--
Iteration 21: [15, 16, 17, 21, 35]
--
Iteration 22: [16, 20, 30]
--
Iteration 23: [28, 38, 42, 48]
--
Iteration 24: [3, 19, 28, 32]
--
Iteration 25: [1, 20, 29, 41]
--
Iteration 26: [10, 16, 27]
--
Iteration 27: [11, 15, 21, 23, 26, 43]
--
Iteration 28: [2, 5, 11, 41, 50]
--
Iteration 29: [11,

### CREATE CORRECT VARIABLES!

Generating Features and Target Variables

In [ ]:
from collections import Counter

X = X_df.to_numpy()
y = ysum_df.to_numpy()
# labels in str form ## sum(axis=1) gives different behaviour, dependent on python version (as it seems)
z = pd.DataFrame(X).iloc[:,:number_of_lines].astype(int).astype(str).sum(axis=1)
l = [] #  list
c = Counter(z)
for k in z:
    #print(k)
    if c[k] == 1:
        l.append(str('G0'))
    else:
        l.append(str(k))

# 1104 G0 values        
#Split data Model ANN 1
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.1, stratify = pd.DataFrame(l)[0], shuffle=True, random_state=42)
    
##Split data Model ANN2
# X_train, X_val, y_train, y_val = train_test_split(X[(np.array(l) != "G0")], y[(np.array(l) != "G0")], train_size=0.1, stratify = pd.DataFrame(l)[(np.array(l) != "G0")][0], shuffle=True,random_state=42)
# X_train = np.append(X_train, X[(np.array(l) == "G0")], axis=0)
# y_train = np.append(y_train, y[(np.array(l) == "G0")], axis=0)

#Split data Model ANN 3
# X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.1,  
#                                                 shuffle=True, random_state=42)

# standardize data
scaler_x = StandardScaler()
scaler_y = StandardScaler()
X_train = np.concatenate((X_train.T[:number_of_lines].T, scaler_x.fit_transform(X_train.T[number_of_lines:].T)), axis=1).astype(float)
X_val = np.concatenate((X_val.T[:number_of_lines].T, scaler_x.transform(X_val.T[number_of_lines:].T)), axis=1).astype(float)# without fit!
y_train = scaler_y.fit_transform(y_train).astype(float)
y_val = scaler_y.transform(y_val).astype(float)


In [ ]:
#%run -i ./v0.py

In [ ]:
z_train = pd.DataFrame(X_train).iloc[:,:number_of_lines].astype(int).astype(str).sum(axis=1) # labels in str form
l_train = [] #  list
c_train = Counter(z_train)
for k in z_train:
    #print(k)
    l_train.append(k)
len(set(l_train))

In [ ]:
pd.DataFrame(X_train)

## Model Preparation

In [ ]:
#%%writefile ./v2.py

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter


torch.manual_seed(555)

# Builds tensors from numpy arrays BEFORE split
X_train_tensor = torch.from_numpy(X_train).float()
X_val_tensor = torch.from_numpy(X_val).float()
y_train_tensor = torch.from_numpy(y_train).float()
y_val_tensor = torch.from_numpy(y_val).float()


# Builds dataset containing ALL data points

dataset_train = TensorDataset(X_train_tensor, y_train_tensor)
dataset_val = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = []
val_loader = []

# from sklearn.model_selection import KFold


# # K-fold Cross Validation model evaluation
# kfold = KFold(n_splits=5, shuffle=True)
# for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset_train)):

#     # Print
#     print(f'FOLD {fold}')
#     print('--------------------------------')

#     # Sample elements randomly from a given list of ids, no replacement.
#     train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
#     test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

#     # Define data loaders for training and testing data in this fold
#     train_loader.append(torch.utils.data.DataLoader(
#                       dataset_train, 
#                       batch_size=256, sampler=train_subsampler))
#     val_loader.append(torch.utils.data.DataLoader(
#                       dataset_train,
#                       batch_size=256, sampler=test_subsampler))

   
train_loader.append(DataLoader(dataset=dataset_train, batch_size=256, shuffle=True)) #batch_size=256
val_loader.append(DataLoader(dataset=dataset_val, batch_size=256))

In [ ]:
#%run -i ./v2.py

## Model Configuration

In [ ]:
#%%writefile ./v3.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Sets learning rate - this is "eta" ~ the "n" like Greek letter
lr = 0.001

torch.manual_seed(42)
# Now we can create a model and send it at once to the device
model = nn.Sequential(nn.Linear(X_train.shape[1], 512), nn.ReLU(),
                      #nn.Dropout(p=0.1),
                      nn.Linear(512, 512), nn.ReLU(),
                      nn.Linear(512, 512), nn.ReLU(),
                      nn.Linear(512,512), nn.ReLU(),
                      nn.Linear(512, 256), nn.ReLU(),
                      nn.Linear(256, y_train.shape[1]) ).to(device)

# Defines an  optimizer to update the parameters (now retrieved directly from the model)
optimizer = optim.Adam(model.parameters(), lr=lr)

# Defines a MSE loss function
loss_fn = nn.MSELoss(reduction='mean')




In [ ]:
#%run -i ./v3.py

## Model Instantiation

In [ ]:
neunet = NeuNet(model, loss_fn, optimizer)
neunet.set_loaders(train_loader[0], val_loader[0])
neunet.set_tensorboard(name="runs", folder = 'machine_learning_tutorial/Test_Grid10')

In [ ]:
print(neunet.model)

## Model Training

In [ ]:
neunet.train(n_epochs = 500)

In [ ]:
#%matplotlib inline
plt.style.use('fivethirtyeight')
fig = neunet.plot_losses()

In [ ]:
print(neunet.val_losses[-1])
print(neunet.losses[-1])

# TensorBoard

In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir machine_learning_tutorial/Test_Grid10

In [ ]:
# prediction of validation points
i = 0
start_t = 50
end_t = start_t + 24*4*5
#new_inputs = torch.tensor(X_val).float()
new_inputs = np.array(X_val)
model.eval()
#pred = model(new_inputs.to(device))
pred = neunet.predict(new_inputs)
orig = y_val
f, (ax) = plt.subplots(1,1, figsize=(15, 10)) 
plt.plot(scaler_y.inverse_transform(orig)[start_t:end_t, i], alpha=.5, linestyle="--", label="correct ENS", lw=1)
plt.plot(scaler_y.inverse_transform(pred)[start_t:end_t, i], alpha=.5, linestyle="-", label="predicted ENS", lw=1)
plt.legend()
plt.show()
MSE = np.mean(scaler_y.inverse_transform(orig)- scaler_y.inverse_transform(pred))**2
print(f"MSE = {MSE}")
print(f"Normalized MSE = {MSE/(scaler_y.inverse_transform(orig).max()-scaler_y.inverse_transform(orig).min())}")
print(f"ENSError = {abs((scaler_y.inverse_transform(pred).mean() - scaler_y.inverse_transform(orig).mean()))/scaler_y.inverse_transform(orig).mean()}")

In [ ]:
# prediction of time series sequences
new_inputs = np.concatenate((X.T[:number_of_lines].T, scaler_x.transform(X.T[number_of_lines:].T)), axis=1).astype(float)

model.eval()
#pred = model(new_inputs.to(device))
pred = neunet.predict(new_inputs)
orig = scaler_y.transform(y)
plt.plot(scaler_y.inverse_transform(orig), alpha=.5, linestyle="--", label="correct ENS", lw=1)
plt.plot(scaler_y.inverse_transform(pred), alpha=.2, linestyle="-", label="predicted ENS", lw=1)
plt.legend()
plt.show()
print(f"MSE = {np.mean(scaler_y.inverse_transform(orig)- scaler_y.inverse_transform(pred))**2}")
print(f"MSEP = {(np.mean(scaler_y.inverse_transform(orig)- scaler_y.inverse_transform(pred))**2)/np.mean(scaler_y.inverse_transform(orig))}")
print(f"ENSError = {abs((scaler_y.inverse_transform(pred).mean() - scaler_y.inverse_transform(orig).mean()))/scaler_y.inverse_transform(orig).mean()}")

In [ ]:

ENSp = np.full((int(len(y)/50), 1), np.nan)
ENSo = np.full((int(len(y)/50), 1), np.nan)
for i, k in enumerate(range(0, len(X), 50), start=0):
    #print(k)
    ENSp[i] = scaler_y.inverse_transform(pred[k:(k+50)]).sum()
    ENSo[i] = scaler_y.inverse_transform(orig[k:(k+50)]).sum()

f, (ax1, ax2) = plt.subplots(1,2, figsize=(15, 10))
ax1.set_xlim(min(np.concatenate([ENSo, ENSp])), max(np.concatenate([ENSo, ENSp])))
ax1.set_ylim(0, 100)
ax2.set_ylim(0, 100)
ax2.set_xlim(min(np.concatenate([ENSo, ENSp])), max(np.concatenate([ENSo, ENSp])))
#ax1.set_ylim([ymin, ymax])
ax1.hist(ENSo, alpha = 0.7)
ax1.set_xlabel("ENS")
ax1.set_ylabel("Density")
ax1.set_title("OPF")
ax2.hist(ENSp, alpha = 0.7)
ax2.set_xlabel("ENS")
ax2.set_ylabel("Density")
ax2.set_title("ANN")
plt.show()

In [ ]:
from scipy.stats import kurtosis, skew
print(f"mean ENS predicted = {ENSp.mean()}")
print(f"mean ENS original = {ENSo.mean()}")
print(f"median ENS predicted = {np.median(ENSp)}")
print(f"median ENS original = {np.median(ENSo)}")
print(f"standard deviation ENS predicted = {ENSp.std()}")
print(f"standard deviation ENS original = {ENSo.std()}")
print(f"kurtosis ENS predicted: {kurtosis(ENSp)}")
print(f"kurtosis ENS original: {kurtosis(ENSo)}")
print(f"skeweness ENS predicted: {skew(ENSp)}")
print(f"skeweness ENS original: {skew(ENSo)}")

# Survivability

In [ ]:
MaxLoad = df_I.iloc[:,42:].sum(axis=1).max()
#MaxLoad = 173.54 #MW (from analysed time series)
crit_load_pu = np.linspace(0,0.45,100)
crit_load = crit_load_pu*MaxLoad
Sp = []
So = []
for i, k in enumerate(range(0, len(X), 50), start=0):
    #print(k)
    Sp.append(scaler_y.inverse_transform(pred[k:(k+50)]).T[0])
    So.append(scaler_y.inverse_transform(orig[k:(k+50)]).T[0])
Sp = np.array(Sp)
So = np.array(So)

q = 1 # quantile deciding the maximum loss of load
#df.iloc[:,5:][(df.type == "load") & (df.field == "max_p_mw")].sum(axis=1).max() # max load
surv_p = np.full(len(crit_load), np.nan)
surv_o = np.full(len(crit_load), np.nan)
for i, c in enumerate(crit_load):
    #surv_p[i] = (Sp.max(axis=1) < c).sum()/240
    surv_p[i] = (np.quantile(Sp, q, axis=1) < c).sum()/240
    #surv_o[i] = (So.max(axis=1) < c).sum()/240
    surv_o[i] = (np.quantile(So, q, axis=1) < c).sum()/240
#plt.plot((1-crit_load_pu), surv)
f, (ax1) = plt.subplots(1,1, figsize=(10, 5))
ax1.plot((1-crit_load_pu), surv_o*100)
ax1.set_xlabel("Supplied Load [pu]")
ax1.set_ylabel("Survivability [%]")
ax1.set_title("Survivability")
ax1.plot((1-crit_load_pu), surv_p*100)
ax1.legend(["OPF", "ANN"], loc ="upper right")
plt.show()

In [ ]:
Sp.max(axis=1).shape

In [ ]:
Sp

# Checkpointing

In [ ]:
neunet.save_checkpoint('model_checkpoint.pth')

## Resuming Training

In [ ]:
#%run -i ./v0.py
#%run -i ./v3.py

In [ ]:
print(model.state_dict())

In [ ]:
new_neunet = NeuNet(model, loss_fn, optimizer)

In [ ]:
new_neunet.load_checkpoint("model_checkpoint.pth")

In [ ]:
print(model.state_dict())

In [ ]:
new_neunet.set_loaders(train_loader[0], val_loader[0])
new_neunet.train(n_epochs=10)

### Plot losses of resumed model

In [ ]:
fig = new_neunet.plot_losses()